In [1]:
import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

# Load from env
load_dotenv()

model = init_chat_model("gpt-4o-mini")

In [7]:
from pathlib import Path

base_path = Path.cwd().parent.parent.parent

workspace_path = base_path / "agents" / "dev" / "workspace"
print("Workspace Path:", workspace_path)

data_path = workspace_path / "data"
print("Data Path:", data_path)

plot_path = workspace_path / "plot"
print("Plot Path:", plot_path)

Workspace Path: /home/afiq/fyp/fafa-repo/backend/app/agents/dev/workspace
Data Path: /home/afiq/fyp/fafa-repo/backend/app/agents/dev/workspace/data
Plot Path: /home/afiq/fyp/fafa-repo/backend/app/agents/dev/workspace/plot


1. check data given the query
2. req for better data given query (optional)
3. plot
4. return plotting path
